In [61]:
import os
import sys

import pandas as pd
import numpy as np

sys.path.insert(0, "/".join(os.path.abspath(os.curdir).split("/")[:-1:]))
from utils.utils import *

pd.set_option('future.no_silent_downcasting', True)

# Dados observatório educacional

## Carregar os dados

In [62]:
df = pd.read_csv("../output/data.csv", low_memory=False)

df.head()

,NO_MUNICIPIO,ANO,TP_DEPENDENCIA,TP_CATEGORIA_ESCOLA_PRIVADA,TP_LOCALIZACAO,TP_LOCALIZACAO_DIFERENCIADA,TP_REGULAMENTACAO,TP_OCUPACAO_PREDIO_ESCOLAR,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,...,QT_MAT_INF_PRE_INT,QT_DOC_INF,QT_DOC_INF_CRE,QT_DOC_INF_PRE,QT_TUR_INF,QT_TUR_INF_CRE,QT_TUR_INF_PRE,IN_PROF_GESTAO,IN_PROF_COORDENADOR,QT_PROF_COORDENADOR
0,Almadina,2015,Municipal,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Sim,Próprio,Não,Sim,...,0,0,0,0,0,0,0,Não aplicável,Não aplicável,Não aplicável
1,Almadina,2015,Municipal,Não aplicável para escolas públicas,Rural,Escola não está em área de localização diferen...,Não,Não aplicável para escolas que não ocupam préd...,Não,Não,...,0,0,0,0,0,0,0,Não aplicável,Não aplicável,Não aplicável
2,Almadina,2015,Municipal,Não aplicável para escolas públicas,Rural,Escola não está em área de localização diferen...,Não,Não aplicável para escolas que não ocupam préd...,Não,Não,...,0,0,0,0,0,0,0,Não aplicável,Não aplicável,Não aplicável
3,Almadina,2015,Municipal,Não aplicável para escolas públicas,Rural,Escola não está em área de localização diferen...,Não,Não aplicável para escolas que não ocupam préd...,Não,Não,...,0,0,0,0,0,0,0,Não aplicável,Não aplicável,Não aplicável
4,Almadina,2015,Municipal,Não aplicável para escolas públicas,Rural,Escola não está em área de localização diferen...,Sim,Cedido,Não,Sim,...,0,0,0,0,0,0,0,Não aplicável,Não aplicável,Não aplicável


## Dados mais específicos da educação infantil dos municípios

In [63]:
columns = ['QT_MAT_BAS_0_3', 'QT_MAT_BAS_4_5', 'QT_MAT_INF', 'QT_DOC_INF']

df_especifico = df[['NO_MUNICIPIO', 'ANO'] + columns].copy()
df_especifico = replace_nao_aplicavel(df_especifico, columns)
df_especifico['QT_ESC'] = df_especifico['QT_MAT_INF'].apply(lambda x: 1)

df_especifico = df_especifico.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(numeric_only=True)
df_especifico

,NO_MUNICIPIO,ANO,QT_MAT_BAS_0_3,QT_MAT_BAS_4_5,QT_MAT_INF,QT_DOC_INF,QT_ESC
0,Almadina,2015,51,152,207,19,18
1,Almadina,2016,56,128,189,17,24
2,Almadina,2017,65,127,207,14,23
3,Almadina,2018,93,130,233,16,24
4,Almadina,2019,92,134,235,14,24
...,...,...,...,...,...,...,...
220,Una,2019,229,509,820,80,87
221,Una,2020,235,472,745,68,32
222,Una,2021,212,465,726,60,87
223,Una,2022,243,472,774,67,87


__Dados extraídos__:

- QT_MAT_BAS_0_3: População de 0 a 3 de idade escolar
- QT_MAT_BAS_4_5: População de 4 a 5 de idade escolar
- QT_MAT_INF: Número geral de matrículas
- QT_DOC_INF: Número geral de docentes
- QT_ESC: Número geral de escolas

## Média de alunos por turma na pré-escola e creche

In [64]:
# Média de alunos por turma
columns = ['QT_MAT_INF', 'QT_MAT_INF_CRE', 'QT_MAT_INF_PRE', 'QT_TUR_INF',
           'QT_TUR_INF_CRE', 'QT_TUR_INF_PRE']

df_mda_alu_tur = df[['NO_MUNICIPIO', 'ANO'] + columns].copy()
df_mda_alu_tur = replace_nao_aplicavel(df_mda_alu_tur, columns)
df_mda_alu_tur['MDA_ALU_TUR_INF'] = get_mean_columns(df_mda_alu_tur, 'QT_MAT_INF', 'QT_TUR_INF')
df_mda_alu_tur['MDA_ALU_TUR_CRE'] = get_mean_columns(df_mda_alu_tur, 'QT_MAT_INF_CRE', 'QT_TUR_INF_CRE')
df_mda_alu_tur['MDA_ALU_TUR_PRE'] = get_mean_columns(df_mda_alu_tur, 'QT_MAT_INF_PRE', 'QT_TUR_INF_PRE')

df_mda_alu_tur = df_mda_alu_tur.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(numeric_only=True)
df_mda_alu_tur

# Média de alunos por turma segundo dependência administrativa
mean_columns = ['QT_MAT_INF', 'QT_TUR_INF']
tmp = get_mean_columns_pivoted(df, mean_columns, 'TP_DEPENDENCIA')
tmp.rename(columns={'Estadual': 'MDA_ALU_TUR_INF_EST', 'Federal': 'MDA_ALU_TUR_INF_FED',
                     'Municipal': 'MDA_ALU_TUR_INF_MUN', 'Privada': 'MDA_ALU_TUR_INF_PRI'}, inplace=True)

# Média de alunos por turma segundo localização
mean_columns = ['QT_MAT_INF', 'QT_TUR_INF']
tmp2 = get_mean_columns_pivoted(df, mean_columns, 'TP_LOCALIZACAO')
tmp2.rename(columns={'Urbana': 'MDA_ALU_TUR_INF_URB', 'Rural': 'MDA_ALU_TUR_INF_RUR'}, inplace=True)

df_mda_alu_tur = pd.merge(df_mda_alu_tur, tmp, on=['NO_MUNICIPIO', 'ANO'], how='left')
df_mda_alu_tur = pd.merge(df_mda_alu_tur, tmp2, on=['NO_MUNICIPIO', 'ANO'], how='left')
df_mda_alu_tur.drop(columns=['QT_MAT_INF_CRE', 'QT_TUR_INF_CRE', 'QT_MAT_INF_PRE',
                             'QT_TUR_INF_PRE'], inplace=True)
df_mda_alu_tur

,NO_MUNICIPIO,ANO,QT_MAT_INF,QT_TUR_INF,MDA_ALU_TUR_INF,MDA_ALU_TUR_CRE,MDA_ALU_TUR_PRE,MDA_ALU_TUR_INF_EST,MDA_ALU_TUR_INF_FED,MDA_ALU_TUR_INF_MUN,MDA_ALU_TUR_INF_PRI,MDA_ALU_TUR_INF_RUR,MDA_ALU_TUR_INF_URB
0,Almadina,2015,207,10,55.750000,24.000000,56.000000,0.0,0.0,23.875000,8.000000,0.000000,20.700000
1,Almadina,2016,189,9,56.250000,26.500000,50.250000,0.0,0.0,22.125000,12.000000,0.000000,21.000000
2,Almadina,2017,207,8,49.500000,33.500000,44.250000,0.0,0.0,25.875000,0.000000,0.000000,24.750000
3,Almadina,2018,233,12,52.650000,43.333333,45.750000,0.0,0.0,22.888889,9.000000,0.000000,19.000000
4,Almadina,2019,235,11,59.750000,46.000000,47.500000,0.0,0.0,26.750000,7.000000,0.000000,21.090909
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Una,2019,820,56,280.608333,33.375000,247.566667,0.0,0.0,16.702128,3.888889,11.541667,16.968750
221,Una,2020,745,50,275.375000,75.375000,228.333333,0.0,0.0,17.119048,3.250000,12.900000,16.233333
222,Una,2021,726,43,265.500000,26.500000,227.000000,0.0,0.0,17.609756,2.000000,13.315789,19.708333
223,Una,2022,774,53,256.638889,30.222222,213.083333,0.0,0.0,16.954545,3.111111,11.095238,16.906250


__Dados extraídos__:

- QT_MAT_INF: Número geral de matrículas
- QT_TUR_INF: Número geral de turmas
- MDA_ALU_TUR_INF: Média geral de alunos por turma
- MDA_ALU_TUR_INF_CRE: Média de alunos por turma em creches
- MDA_ALU_TUR_INF_PRE: Média de alunos por turma em pré-escolas
- MDA_ALU_TUR_INF_EST: Média de alunos por turma em escolas estaduais
- MDA_ALU_TUR_INF_FED: Média de alunos por turma em escolas federais
- MDA_ALU_TUR_INF_MUN: Média de alunos por turma em escolas municipais
- MDA_ALU_TUR_INF_PRI: Média de alunos por turma em escolas privadas
- MDA_ALU_TUR_INF_RUR: Média de alunos por turma em localidades rurais
- MDA_ALU_TUR_INF_URB: Média de alunos por turma em localidades urbanas

## Número de estabelecimentos de educação infantil (creches)

In [65]:
columns = ["QT_MAT_INF_CRE"]

df_estab_cre = df[["NO_MUNICIPIO", "ANO"] + columns].copy()

df_estab_cre = replace_nao_aplicavel(df_estab_cre, columns)
df_estab_cre["QT_ESC_INF_CRE"] = df_estab_cre["QT_MAT_INF_CRE"].apply(
    lambda x: 1 if x > 0 else 0
)

df_estab_cre = df_estab_cre.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(
    numeric_only=True
).drop(columns="QT_MAT_INF_CRE")

tmp = get_qt_estabelecimento_columns_pivoted(df, 'QT_MAT_INF_CRE', 'TP_DEPENDENCIA')
tmp.rename(columns={'Estadual': 'QT_ESC_INF_CRE_EST', 'Federal': 'QT_ESC_INF_CRE_FED',
                        'Municipal': 'QT_ESC_INF_CRE_MUN', 'Privada': 'QT_ESC_INF_CRE_PRI'}, inplace=True)

df_estab_cre = pd.merge(df_estab_cre, tmp, on=['NO_MUNICIPIO', 'ANO'], how='left')

# Número de pré-escolas por localização
tmp = get_qt_estabelecimento_columns_pivoted(df, 'QT_MAT_INF_CRE', 'TP_LOCALIZACAO')
tmp.rename(columns={'Urbana': 'QT_INF_CRE_URB', 'Rural': 'QT_INF_CRE_RUR'}, inplace=True)

df_estab_cre = pd.merge(df_estab_cre, tmp, on=['NO_MUNICIPIO', 'ANO'], how='left')
df_estab_cre

,NO_MUNICIPIO,ANO,QT_ESC_INF_CRE,QT_ESC_INF_CRE_EST,QT_ESC_INF_CRE_FED,QT_ESC_INF_CRE_MUN,QT_ESC_INF_CRE_PRI,QT_INF_CRE_RUR,QT_INF_CRE_URB
0,Almadina,2015,1,0,0,1,0,0,1
1,Almadina,2016,1,0,0,1,0,0,1
2,Almadina,2017,2,0,0,2,0,1,1
3,Almadina,2018,2,0,0,1,1,0,2
4,Almadina,2019,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...
220,Una,2019,6,0,0,3,3,2,4
221,Una,2020,10,0,0,8,2,7,3
222,Una,2021,6,0,0,6,0,5,1
223,Una,2022,8,0,0,6,2,5,3


__Dados extraídos__:

- QT_ESC_INF_CRE: Número geral de matrículas
- QT_ESC_INF_CRE_EST: Número de creches estaduais
- QT_ESC_INF_CRE_FED: Número de creches federais
- QT_ESC_INF_CRE_MUN: Número de creches municipais
- QT_ESC_INF_CRE_PRI: Número de creches privadas
- QT_ESC_INF_CRE_RUR: Número de creches em localidade rural
- QT_ESC_INF_CRE_URB: Número de creches em localidade urbana

## Número de estabelecimentos de educação infantil (pré-escola)

In [66]:
columns = ["QT_MAT_INF_PRE"]

df_estab_pre = df[["NO_MUNICIPIO", "ANO"] + columns].copy()
df_estab_pre = replace_nao_aplicavel(df_estab_pre, columns)
df_estab_pre["QT_INF_PRE"] = df_estab_pre["QT_MAT_INF_PRE"].apply(
    lambda x: 1 if x > 0 else 0
)
df_estab_pre = df_estab_pre.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(
    numeric_only=True
).drop(columns="QT_MAT_INF_PRE")

# Número de pré-escolas por dependência administrativa
tmp = get_qt_estabelecimento_columns_pivoted(df, 'QT_MAT_INF_PRE', 'TP_DEPENDENCIA')
tmp.rename(columns={'Estadual': 'QT_INF_PRE_EST', 'Federal': 'QT_INF_PRE_FED',
                        'Municipal': 'QT_INF_PRE_MUN', 'Privada': 'QT_INF_PRE_PRI'}, inplace=True)

df_estab_pre = pd.merge(df_estab_pre, tmp, on=['NO_MUNICIPIO', 'ANO'], how='left')

# Número de pré-escolas por localização
tmp = get_qt_estabelecimento_columns_pivoted(df, 'QT_MAT_INF_PRE', 'TP_LOCALIZACAO')
tmp.rename(columns={'Urbana': 'QT_INF_PRE_URB', 'Rural': 'QT_INF_PRE_RUR'}, inplace=True)

df_estab_pre = pd.merge(df_estab_pre, tmp, on=['NO_MUNICIPIO', 'ANO'], how='left')
df_estab_pre

,NO_MUNICIPIO,ANO,QT_INF_PRE,QT_INF_PRE_EST,QT_INF_PRE_FED,QT_INF_PRE_MUN,QT_INF_PRE_PRI,QT_INF_PRE_RUR,QT_INF_PRE_URB
0,Almadina,2015,3,0,0,2,1,0,3
1,Almadina,2016,3,0,0,2,1,0,3
2,Almadina,2017,4,0,0,4,0,2,2
3,Almadina,2018,5,0,0,4,1,2,3
4,Almadina,2019,5,0,0,4,1,2,3
...,...,...,...,...,...,...,...,...,...
220,Una,2019,25,0,0,22,3,18,7
221,Una,2020,22,0,0,19,3,15,7
222,Una,2021,19,0,0,18,1,14,5
223,Una,2022,21,0,0,19,2,15,6


__Dados extraídos__:

- QT_INF_PRE: Número geral de matrículas
- QT_INF_PRE_EST: Número de pré-escolas estaduais
- QT_INF_PRE_FED: Número de pré-escolas federais
- QT_INF_PRE_MUN: Número de pré-escolas municipais
- QT_INF_PRE_PRI: Número de pré-escolas privadas
- QT_ESC_INF_PRE_RUR: Número de pré-escolas em localidade rural
- QT_ESC_INF_PRE_URB: Número de pré-escolas em localidade urbana

## Número de matrículas em creches

In [67]:
columns = ["QT_MAT_INF_CRE"]

df_mat_cre = df[["NO_MUNICIPIO", "ANO"] + columns].copy()
df_mat_cre = replace_nao_aplicavel(df_mat_cre, columns)
df_mat_cre = df_mat_cre.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(
    numeric_only=True
)
tmp = df[["NO_MUNICIPIO", "ANO", "QT_MAT_INF_CRE", "TP_DEPENDENCIA"]].copy()
tmp = replace_nao_aplicavel(tmp, ["QT_MAT_INF_CRE"])
tmp = tmp.groupby(by=["NO_MUNICIPIO", "ANO", "TP_DEPENDENCIA"], as_index=False).sum(
    numeric_only=True
)
tmp = tmp.pivot_table(index=["NO_MUNICIPIO", "ANO"], columns="TP_DEPENDENCIA",
                      values="QT_MAT_INF_CRE").reset_index()
tmp.fillna(0, inplace=True)
tmp[['Estadual', 'Federal', 'Municipal', 'Privada']] = tmp[['Estadual', 'Federal',
                                                            'Municipal', 'Privada']].astype(int)
tmp.rename(columns={'Estadual': 'QT_MAT_INF_CRE_EST', 'Federal': 'QT_MAT_INF_CRE_FED',
                    'Municipal': 'QT_MAT_INF_CRE_MUN', 'Privada': 'QT_MAT_INF_CRE_PRI'}, inplace=True)

df_mat_cre = pd.merge(df_mat_cre, tmp, on=['NO_MUNICIPIO', 'ANO'], how='left')
df_mat_cre

,NO_MUNICIPIO,ANO,QT_MAT_INF_CRE,QT_MAT_INF_CRE_EST,QT_MAT_INF_CRE_FED,QT_MAT_INF_CRE_MUN,QT_MAT_INF_CRE_PRI
0,Almadina,2015,48,0,0,48,0
1,Almadina,2016,53,0,0,53,0
2,Almadina,2017,69,0,0,69,0
3,Almadina,2018,102,0,0,88,14
4,Almadina,2019,92,0,0,92,0
...,...,...,...,...,...,...,...
220,Una,2019,231,0,0,224,7
221,Una,2020,276,0,0,269,7
222,Una,2021,226,0,0,226,0
223,Una,2022,265,0,0,259,6


## Número de matrículas em pré-escolas

In [68]:
columns = ["QT_MAT_INF_PRE"]

df_mat_pre = df[["NO_MUNICIPIO", "ANO"] + columns].copy()
df_mat_pre = replace_nao_aplicavel(df_mat_pre, columns)
df_mat_pre = df_mat_pre.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(
    numeric_only=True
)
tmp = df[["NO_MUNICIPIO", "ANO", "QT_MAT_INF_PRE", "TP_DEPENDENCIA"]].copy()
tmp = replace_nao_aplicavel(tmp, ["QT_MAT_INF_PRE"])
tmp = tmp.groupby(by=["NO_MUNICIPIO", "ANO", "TP_DEPENDENCIA"], as_index=False).sum(
    numeric_only=True
)
tmp = tmp.pivot_table(index=["NO_MUNICIPIO", "ANO"], columns="TP_DEPENDENCIA",
                      values="QT_MAT_INF_PRE").reset_index()
tmp.fillna(0, inplace=True)
tmp[['Estadual', 'Federal', 'Municipal', 'Privada']] = tmp[['Estadual', 'Federal',
                                                            'Municipal', 'Privada']].astype(int)
tmp.rename(columns={'Estadual': 'QT_MAT_INF_PRE_EST', 'Federal': 'QT_MAT_INF_PRE_FED',
                    'Municipal': 'QT_MAT_INF_PRE_MUN', 'Privada': 'QT_MAT_INF_PRE_PRI'}, inplace=True)

df_mat_pre = pd.merge(df_mat_pre, tmp, on=['NO_MUNICIPIO', 'ANO'], how='left')
df_mat_pre

,NO_MUNICIPIO,ANO,QT_MAT_INF_PRE,QT_MAT_INF_PRE_EST,QT_MAT_INF_PRE_FED,QT_MAT_INF_PRE_MUN,QT_MAT_INF_PRE_PRI
0,Almadina,2015,159,0,0,143,16
1,Almadina,2016,136,0,0,124,12
2,Almadina,2017,138,0,0,138,0
3,Almadina,2018,131,0,0,118,13
4,Almadina,2019,143,0,0,122,21
...,...,...,...,...,...,...,...
220,Una,2019,589,0,0,561,28
221,Una,2020,469,0,0,450,19
222,Una,2021,500,0,0,496,4
223,Una,2022,509,0,0,487,22


## Infraestrutura

In [69]:
columns = ['IN_INF_PRE', 'IN_INF_CRE', 'IN_AGUA_FILTRADA', 'IN_AGUA_POTAVEL', 'IN_BANHEIRO_PNE',
           'IN_BANHEIRO_EI', 'IN_BIBLIOTECA', 'IN_LIXO_SERVICO_COLETA', 'IN_COZINHA',
           'IN_REFEITORIO', 'IN_PARQUE_INFANTIL', 'IN_QUADRA_ESPORTES',
           'IN_SALA_DIRETORIA', 'IN_BIBLIOTECA_SALA_LEITURA', 'IN_SALA_PROFESSOR',
           'IN_ENERGIA_INEXISTENTE', "IN_AGUA_INEXISTENTE", "IN_ESGOTO_INEXISTENTE"]
columns_to_drop = columns

df_infra = df[['NO_MUNICIPIO', 'ANO'] + columns].copy()
df_infra = replace_sim_nao_to_binary(df_infra, columns)

df_infra['QT_ESC'] = df_infra['IN_INF_PRE'].apply(lambda x: 1)
df_infra['QT_ESC_INF_PRE'] = df_infra['IN_INF_CRE']
df_infra['QT_ESC_INF_CRE'] = df_infra['IN_INF_PRE']
columns_to_drop += ['QT_ESC', 'QT_ESC_INF_PRE', 'QT_ESC_INF_CRE']

water_qualities = ['POTAVEL', 'FILTRADA']
bathroom_types = ['PNE', 'EI']

for water_quality in water_qualities:
    df_infra[f'IN_AGUA_{water_quality}_CRE'] = df_infra.apply(
        lambda x: 1 if x[f'IN_AGUA_{water_quality}'] == 1
        and x['IN_INF_CRE'] == 1 else 0, axis=1
    )
    df_infra[f'IN_AGUA_{water_quality}_PRE'] = df_infra.apply(
        lambda x: 1 if x[f'IN_AGUA_{water_quality}'] == 1
        and x['IN_INF_PRE'] == 1 else 0, axis=1
    )

for bathroom_type in bathroom_types:
    df_infra[f'IN_BANHEIRO_{bathroom_type}_CRE'] = df_infra.apply(
        lambda x: 1 if x[f'IN_BANHEIRO_{bathroom_type}'] == 1
        and x['IN_INF_CRE'] == 1 else 0, axis=1
    )
    df_infra[f'IN_BANHEIRO_{bathroom_type}_PRE'] = df_infra.apply(
        lambda x: 1 if x[f'IN_BANHEIRO_{bathroom_type}'] == 1
        and x['IN_INF_PRE'] == 1 else 0, axis=1
    )

df_infra = df_infra.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(numeric_only=True)
df_infra.head()

,NO_MUNICIPIO,ANO,IN_INF_PRE,IN_INF_CRE,IN_AGUA_FILTRADA,IN_AGUA_POTAVEL,IN_BANHEIRO_PNE,IN_BANHEIRO_EI,IN_BIBLIOTECA,IN_LIXO_SERVICO_COLETA,...,QT_ESC_INF_PRE,QT_ESC_INF_CRE,IN_AGUA_POTAVEL_CRE,IN_AGUA_POTAVEL_PRE,IN_AGUA_FILTRADA_CRE,IN_AGUA_FILTRADA_PRE,IN_BANHEIRO_PNE_CRE,IN_BANHEIRO_PNE_PRE,IN_BANHEIRO_EI_CRE,IN_BANHEIRO_EI_PRE
0,Almadina,2015,3,1,12,0,1,2,2,8,...,1,3,0,0,1,3,0,0,1,2
1,Almadina,2016,3,1,12,0,2,2,2,8,...,1,3,0,0,1,3,0,0,1,2
2,Almadina,2017,0,0,12,0,2,1,1,7,...,0,0,0,0,0,0,0,0,0,0
3,Almadina,2018,5,2,13,0,2,2,1,8,...,2,5,0,0,2,5,0,0,2,2
4,Almadina,2019,5,1,0,8,2,2,0,8,...,1,5,1,3,0,0,0,0,1,2


In [70]:
general_columns_prop = ['BIBLIOTECA', 'LIXO_SERVICO_COLETA', 'REFEITORIO',
                        'COZINHA', 'PARQUE_INFANTIL', 'QUADRA_ESPORTES', 'SALA_DIRETORIA',
                        'BIBLIOTECA_SALA_LEITURA', 'SALA_PROFESSOR', 'ENERGIA_INEXISTENTE',
                        "AGUA_INEXISTENTE", "ESGOTO_INEXISTENTE"]

for column in general_columns_prop:
    df_infra[f'PROP_{column}'] = get_prop_columns(df_infra, f'IN_{column}', 'QT_ESC')
    df_infra[f'PROP_{column}_INF_CRE'] = get_prop_columns(df_infra, f'IN_{column}', 'QT_ESC_INF_CRE')
    df_infra[f'PROP_{column}_INF_PRE'] = get_prop_columns(df_infra, f'IN_{column}', 'QT_ESC_INF_PRE')

    columns_to_drop += [f'IN_{column}']

for water_quality in water_qualities:
    df_infra[f'PROP_AGUA_{water_quality}'] = get_prop_columns(
        df_infra,f'IN_AGUA_{water_quality}', 'QT_ESC'
        )
    df_infra[f'PROP_AGUA_{water_quality}_INF_CRE'] = get_prop_columns(
        df_infra, f'IN_AGUA_{water_quality}_CRE', 'QT_ESC_INF_PRE'
        )
    df_infra[f'PROP_AGUA_{water_quality}_INF_PRE'] = get_prop_columns(
        df_infra, f'IN_AGUA_{water_quality}_PRE', 'QT_ESC_INF_CRE'
        )

    columns_to_drop += [f'IN_AGUA_{water_quality}_CRE', f'IN_AGUA_{water_quality}_PRE']

for bathroom_type in bathroom_types:
    df_infra[f'PROP_BANHEIRO_{bathroom_type}'] = get_prop_columns(
        df_infra,f'IN_BANHEIRO_{bathroom_type}', 'QT_ESC'
        )
    df_infra[f'PROP_BANHEIRO_{bathroom_type}_INF_CRE'] = get_prop_columns(
        df_infra, f'IN_BANHEIRO_{bathroom_type}_CRE', 'QT_ESC_INF_PRE'
        )
    df_infra[f'PROP_BANHEIRO_{bathroom_type}_INF_PRE'] = get_prop_columns(
        df_infra, f'IN_BANHEIRO_{bathroom_type}_PRE', 'QT_ESC_INF_CRE'
        )

    columns_to_drop += [f'IN_BANHEIRO_{bathroom_type}_CRE', f'IN_BANHEIRO_{bathroom_type}_PRE']

df_infra.drop(columns=columns_to_drop, inplace=True)

df_infra

,NO_MUNICIPIO,ANO,PROP_BIBLIOTECA,PROP_BIBLIOTECA_INF_CRE,PROP_BIBLIOTECA_INF_PRE,PROP_LIXO_SERVICO_COLETA,PROP_LIXO_SERVICO_COLETA_INF_CRE,PROP_LIXO_SERVICO_COLETA_INF_PRE,PROP_REFEITORIO,PROP_REFEITORIO_INF_CRE,...,PROP_AGUA_POTAVEL_INF_PRE,PROP_AGUA_FILTRADA,PROP_AGUA_FILTRADA_INF_CRE,PROP_AGUA_FILTRADA_INF_PRE,PROP_BANHEIRO_PNE,PROP_BANHEIRO_PNE_INF_CRE,PROP_BANHEIRO_PNE_INF_PRE,PROP_BANHEIRO_EI,PROP_BANHEIRO_EI_INF_CRE,PROP_BANHEIRO_EI_INF_PRE
0,Almadina,2015,0.11,0.67,0.00,0.44,0.00,0.0,0.06,0.33,...,0.0,0.67,1.0,1.0,0.06,0.00,0.00,0.11,1.00,0.67
1,Almadina,2016,0.08,0.67,0.00,0.33,0.00,0.0,0.04,0.33,...,0.0,0.50,1.0,1.0,0.08,0.00,0.00,0.08,1.00,0.67
2,Almadina,2017,0.04,0.00,0.00,0.30,0.00,0.0,0.04,0.00,...,0.0,0.52,0.0,0.0,0.09,0.00,0.00,0.04,0.00,0.00
3,Almadina,2018,0.04,0.20,0.50,0.33,0.00,0.0,0.04,0.20,...,0.0,0.54,1.0,1.0,0.08,0.00,0.00,0.08,1.00,0.40
4,Almadina,2019,0.00,0.00,0.00,0.33,0.00,0.0,0.04,0.20,...,0.6,0.00,0.0,0.0,0.08,0.00,0.00,0.08,1.00,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Una,2019,0.05,0.16,0.67,0.22,0.76,0.0,0.06,0.20,...,1.0,0.00,0.0,0.0,0.08,0.33,0.20,0.05,0.33,0.12
221,Una,2020,0.12,0.18,0.40,0.62,0.91,0.0,0.16,0.23,...,1.0,0.00,0.0,0.0,0.28,0.40,0.32,0.12,0.20,0.14
222,Una,2021,0.05,0.21,0.67,0.23,0.00,0.0,0.06,0.26,...,1.0,0.00,0.0,0.0,0.10,0.50,0.37,0.05,0.17,0.11
223,Una,2022,0.05,0.19,0.50,0.23,0.95,0.0,0.06,0.24,...,1.0,0.00,0.0,0.0,0.10,0.38,0.29,0.06,0.25,0.14
